## 폴백(fallback)

### LLM API Error 에 대처 방법

중요: 기본적으로 많은 LLM 래퍼(wrapper)는 오류를 포착하고 재시도합니다. fallback 을 사용할 때는 이러한 기본 동작을 해제하는 것이 좋습니다. 그렇지 않으면 첫 번째 래퍼가 계속 재시도하고 실패하지 않을 것입니다.

In [1]:
# !pip install -qU langchain langchain-openai

In [2]:
from langchain_anthropic import ChatAnthropic
from langchain_openai import ChatOpenAI

In [3]:
from unittest.mock import patch

import httpx
from openai import RateLimitError

request = httpx.Request("GET", "/")  # GET 요청을 생성합니다.
response = httpx.Response(
    200, request=request
)  # 200 상태 코드와 함께 응답을 생성합니다.
# "rate limit" 메시지와 응답 및 빈 본문을 포함하는 RateLimitError를 생성합니다.
error = RateLimitError("rate limit", response=response, body="")

- openai_llm 변수에 ChatOpenAI 객체를 생성하고, max_retries 매개변수를 0으로 설정하여 API 호출비용 제한 등으로 인한 재시도를 방지 합니다.

- with_fallbacks 메서드를 사용하여 anthropic_llm을 fallback LLM으로 설정하고, 이를 llm 변수에 할당합니다.

In [5]:
from dotenv import load_dotenv

load_dotenv()

True

In [6]:
# OpenAI의 ChatOpenAI 모델을 사용하여 openai_llm 객체를 생성합니다.
# max_retries를 0으로 설정하여 속도 제한 등으로 인한 재시도를 방지합니다.
openai_llm = ChatOpenAI(max_retries=0)

# Anthropic의 ChatAnthropic 모델을 사용하여 anthropic_llm 객체를 생성합니다.
anthropic_llm = ChatAnthropic(model="claude-3-opus-20240229")

# openai_llm을 기본으로 사용하고, 실패 시 anthropic_llm을 대체로 사용하도록 설정합니다.
llm = openai_llm.with_fallbacks([anthropic_llm])

In [7]:
# OpenAI LLM을 먼저 사용하여 오류가 발생하는 것을 보여줍니다.
with patch("openai.resources.chat.completions.Completions.create", side_effect=error):
    try:
        # "닭이 길을 건넌 이유는 무엇일까요?"라는 질문을 OpenAI LLM에 전달합니다.
        print(openai_llm.invoke("Why did the chicken cross the road?"))
    except RateLimitError:
        # 오류가 발생하면 오류를 출력합니다.
        print("에러 발생")

에러 발생


In [8]:
# OpenAI API 호출 시 에러가 발생하는 경우 Anthropic 으로 대체하는 코드
with patch("openai.resources.chat.completions.Completions.create", side_effect=error):
    try:
        # "대한민국의 수도는 어디야?"라는 질문을 언어 모델에 전달하여 응답을 출력합니다.
        print(llm.invoke("대한민국의 수도는 어디야?"))
    except RateLimitError:
        # RateLimitError가 발생하면 "에러 발생"를 출력합니다.
        print("에러 발생")

에러 발생


In [9]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "질문에 짧고 간결하게 답변해 주세요.",  # 시스템 역할 설명
        ),
        ("human", "{country} 의 수도는 어디입니까?"),  # 사용자 질문 템플릿
    ]
)
chain = prompt | llm  # 프롬프트와 언어 모델을 연결하여 체인 생성
# chain = prompt | ChatOpenAI() # 이 코드이 주석을 풀고 실행하면 "오류 발생" 문구가 출력됩니다.
with patch("openai.resources.chat.completions.Completions.create", side_effect=error):
    try:
        print(chain.invoke({"country": "대한민국"}))  # 체인을 호출하여 결과 출력
    except RateLimitError:  # API 비용 제한 오류 처리
        print("오류 발생")

오류 발생


### 처리해야할 오류를 구체적으로 명시한 경우

In [10]:
llm = openai_llm.with_fallbacks(
    # 대체 LLM으로 anthropic_llm을 사용하고, 예외 처리할 대상으로 KeyboardInterrupt를 지정합니다.
    [anthropic_llm],
    exceptions_to_handle=(KeyboardInterrupt,),  # 예외 처리 대상을 지정합니다.
)

# 프롬프트와 LLM을 연결하여 체인을 생성합니다.
chain = prompt | llm
with patch("openai.resources.chat.completions.Completions.create", side_effect=error):
    try:
        # 체인을 호출하여 결과를 출력합니다.
        print(chain.invoke({"country": "대한민국"}))
    except RateLimitError:
        # RateLimitError 예외가 발생하면 "오류 발생"를 출력합니다.
        print("오류 발생")

오류 발생


### fallback 에 여러 모델을 순차적으로 지정

In [11]:
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

# 프롬프트 생성
prompt_template = (
    "질문에 짧고 간결하게 답변해 주세요.\n\nQuestion:\n{question}\n\nAnswer:"
)
prompt = PromptTemplate.from_template(prompt_template)

In [12]:
# 여기서는 쉽게 오류를 발생시킬 수 있는 잘못된 모델 이름을 사용하여 체인을 생성할 것입니다.
chat_model = ChatOpenAI(model_name="gpt-fake")
bad_chain = prompt | chat_model

In [13]:
# fallback 체인을 생성합니다.
fallback_chain1 = prompt | ChatOpenAI(model="gpt-3.6-turbo") # 오류
fallback_chain2 = prompt | ChatOpenAI(model="gpt-3.5-turbo") # 정상
fallback_chain3 = prompt | ChatOpenAI(model="gpt-4-turbo-preview") # 정상

In [14]:
# 두 개의 체인을 결합하여 최종 체인을 생성합니다.
chain = bad_chain.with_fallbacks(
    [fallback_chain1, fallback_chain2, fallback_chain3])
# 생성된 체인을 호출하여 입력값을 전달합니다.
chain.invoke({"question": "대한민국의 수도는 어디야?"})

AIMessage(content='서울입니다.', response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 46, 'total_tokens': 51}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-46ca2c49-0943-4d25-8263-91c16f408e5d-0', usage_metadata={'input_tokens': 46, 'output_tokens': 5, 'total_tokens': 51})